In [1]:
from Manager import *
from VisionUtils import *
tf.enable_eager_execution()

In [2]:
IMG_RT_PATH = 'data/images/'
FEATURE_RT_PATH = "data/image_features/"
FEATURE_AUG_RT_PATH = "data/image_aug_features/"
IMG_AUG_RT_PATH = 'data/images_aug/'

#img_list = pd.read_csv(IMG_RT_PATH + "test_list.TXT")
feature_extractor = load_model("Models\FaceNet/Facenet_keras.h5")
face_detector = MTCNN()

W0828 00:30:11.108951 11672 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W0828 00:30:11.113940 11672 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\mtcnn\mtcnn.py:187: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0828 00:30:11.115935 11672 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\mtcnn\mtcnn.py:193: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0828 00:30:11.137899 11672 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\mtcnn\network.py:43: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0828 00:30:11.137899 11672 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\mtcnn\layer_factory.py:88: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 00:30:11.13

In [3]:
import os 
img_path_list = []
folder_list = os.listdir(IMG_RT_PATH)
for FOLDER in folder_list:
    image_list_names = os.listdir(IMG_RT_PATH + FOLDER)
    for NAME in image_list_names:
        img_path_list.append(FOLDER+'/'+NAME)

In [4]:
img_list = pd.DataFrame({"path": img_path_list})

In [5]:
iman = ImageManager(
    imgpaths = img_list,
    feature_extractor = feature_extractor,
    face_detector =  face_detector, 
    IMG_RTPATH = IMG_RT_PATH,
)

In [6]:
iman.face_detection_check()

KeyboardInterrupt: 

iman.augment_images()

iman.extract_all_features(augment = True) ### dont run

iman.face_detection_check()

iman.update_metainfo()
iman.balance_random_sample(100)
iman.split(train_size = 0.75)

len( iman.xtrainp["path1"].values)

feat_list1 =[]
feat_list2 = []
index = []
i = 0
y_aug =  []
t1 = []
for PATH1, PATH2 in zip(iman.xtrainp["path1"].values, iman.xtrainp["path2"].values):
    if ((os.path.isfile(FEATURE_AUG_RT_PATH + PATH1[:-4]+".npy") ==True) and 
    (os.path.isfile(FEATURE_AUG_RT_PATH + PATH2[:-4]+".npy") == True)):
        feat_list1.append(np.load(FEATURE_AUG_RT_PATH + PATH1[:-4]+".npy"))
        feat_list2.append(np.load(FEATURE_AUG_RT_PATH + PATH2[:-4]+".npy"))
        
        if (PATH1[:7] == PATH2[:7]):
            t1.append(1)
        else:
            t1.append(0)
        
    i = i + 1

train_AUG_feat1 = np.vstack(feat_list1)
train_AUG_feat2 = np.vstack(feat_list2)

del feat_list1
del feat_list2

feat_list1 =[]
feat_list2 = []
index = []
i = 0
y_aug =  []
t1 = []
for PATH1, PATH2 in zip(iman.xtrainp["path1"].values, iman.xtrainp["path2"].values):
    if ((os.path.isfile(FEATURE_AUG_RT_PATH + PATH1[:-4]+".npy") ==True) and 
    (os.path.isfile(FEATURE_AUG_RT_PATH + PATH2[:-4]+".npy") == True)):
        feat_list1.append(np.load(FEATURE_AUG_RT_PATH + PATH1[:-4]+".npy"))
        feat_list2.append(np.load(FEATURE_AUG_RT_PATH + PATH2[:-4]+".npy"))
        
        if (PATH1[:7] == PATH2[:7]):
            t1.append(1)
        else:
            t1.append(0)
        
    i = i + 1

train_AUG_feat1 = np.vstack(feat_list1)
train_AUG_feat2 = np.vstack(feat_list2)

del feat_list1
del feat_list2

feat_list1 =[]
feat_list2 = []
index = []
i = 0
y_aug =  []
t2 = []
for PATH1, PATH2 in zip(iman.xtrainp["path1"].values, iman.xtrainp["path2"].values):
    if ((os.path.isfile(FEATURE_RT_PATH + PATH1[:-4]+".npy") ==True) and 
    (os.path.isfile(FEATURE_RT_PATH + PATH2[:-4]+".npy") == True)):
        feat_list1.append(np.load(FEATURE_RT_PATH + PATH1[:-4]+".npy"))
        feat_list2.append(np.load(FEATURE_RT_PATH + PATH2[:-4]+".npy"))
        
        if (PATH1[:7] == PATH2[:7]):
            t2.append(1)
        else:
            t2.append(0)
        
    i = i + 1

train_feat1 = np.vstack(feat_list1)
train_feat2 = np.vstack(feat_list2)

del feat_list1
del feat_list2

feat_list1 =[]
feat_list2 = []
index = []
i = 0
y =  []
t3 = []
iman.ytestp
for PATH1, PATH2 in zip(iman.xtestp["path1"].values, iman.xtestp["path2"].values):
    if ((os.path.isfile(FEATURE_RT_PATH + PATH1[:-4]+".npy") ==True) and 
    (os.path.isfile(FEATURE_RT_PATH + PATH2[:-4]+".npy") == True)):
        feat_list1.append(np.load(FEATURE_RT_PATH + PATH1[:-4]+".npy"))
        feat_list2.append(np.load(FEATURE_RT_PATH + PATH2[:-4]+".npy"))
        y.append(iman.ytestp[i])
        if (PATH1[:7] == PATH2[:7]):
            t3.append(1)
        else:
            t3.append(0)
        
    i = i + 1

test_feat1 = np.vstack(feat_list1)
test_feat2 = np.vstack(feat_list2)

del feat_list1
del feat_list2

labels_aug = np.array(t1 + t2 + t3)

len(labels_aug)

labels_aug

len(train_aug_feat1) + len(train_feat1) + len(test_feat1)

import pandas as pd
np.save("features/train_aug_feat1.npy", train_aug_feat1)
np.save("features/train_aug_feat2.npy", train_aug_feat2)

np.save("features/train_feat1.npy", train_feat1)
np.save("features/train_feat2.npy", train_feat2)

np.save("features/test_feat1.npy", test_feat1)
np.save("features/test_feat2.npy", test_feat2)

np.save("features/labels_aug.npy", labels_aug)

paths = iman.get_sample_image_paths()
paths_df = pd.DataFrame(paths)
paths_df.to_csv("features/image_paths.csv")

In [1]:
import imgaug